In [ ]:
def f1(p, r):
    return 2 * (p * r) / (p+r)
    
print(f1(.943, .945))
print(f1(0.632926229466665, 0.5877095036192854))
    

In [ ]:
import pbs3

In [ ]:
pbs3.pwd().stdout

In [ ]:
import z5py
import numpy
import matplotlib.pyplot as plt

from pathlib import Path

In [ ]:
# fname = "FishStaticLS_488nm_8f43dd13abf527f124bc0463bd5638ed2754dbe471a80a6c5867a675.n5"
# fname = "F02-FishDynamicLF_561nm_TP00_15167be77831e5760090446ed0baf3e249d5be4f3bb356c45ae6e293.n5"
fname = "F02-FishStaticLS_488nm_8f43dd13abf527f124bc0463bd5638ed2754dbe471a80a6c5867a675.n5"
# fname = "FishStaticLS_488nm_714734d3bf750d081c8025513aea2eec2b56fad6cdbd24154174f70a.n5"
# fname = "platyn_neuro-3_60e460ce9d1ec0e045b7d0d61ed73df0e687227d06453035ab4cdbdb.n5"
# fname = "3_60e460ce9d1ec0e045b7d0d61ed73df0e687227d06453035ab4cdbdb.n5"
data_path = "/g/kreshuk/beuttenm/repos/lensletnet/data/"
assert Path(data_path + fname).exists(), data_path + fname
file = z5py.File(data_path + fname, mode="r")
x, y = file["x"], file["y"]

In [ ]:
y_show = numpy.array(y[0, 0])[0, 0]
print(x.shape, y.shape)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(y_show.max(axis=1))
plt.show()
plt.figure(figsize=(20, 20))
plt.imshow(y_show.max(axis=2))
plt.show()

In [ ]:
import sys
import numpy
import torch
from time import perf_counter

sys.path.append("..")
from lnet.models import M13dout, M12dout
from lnet.utils.transforms import Lightfield2Channel, channel_from_lightfield

from torch.utils.data import DataLoader, Dataset

In [ ]:
class TestDS(Dataset):
    def __init__(self, *seqs):
        self._len = len(seqs[0])
        assert all([len(s) == len(self) for s in seqs])
        self.seqs = seqs
        
    def __len__(self):
        return self._len

    def __getitem__(self, i: int):
        return tuple([s[i] for s in self.seqs])

In [ ]:
ds = TestDS(x, y)

In [ ]:
start = perf_counter()
x[10], y[10]
print('dur', perf_counter() - start)

In [ ]:
dl = DataLoader(ds, num_workers=20, pin_memory=True)

In [ ]:
start = perf_counter()
dl_iter = iter(dl)
print('iter', perf_counter() - start)
start = perf_counter()
for xx, yy in dl:
    xx.shape
    break
print('dur', perf_counter() - start)


In [ ]:
torch.cuda.device_count()

In [ ]:
nx = numpy.array([channel_from_lightfield(xx) for xx in x[:100]])

In [ ]:
nx.nbytes / 1024 / 1024

In [ ]:
model = M13dout(z_out=35, nnum=19)
model.cuda()

In [ ]:
tx = torch.from_numpy(nx)

In [ ]:
tx = tx.cuda()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
out = ""
n = 1
start = perf_counter()
for xx in tx[:n]:
    ipt = xx[None]
    out += f"{model(ipt)[0].shape}\n"
    

dur = perf_counter() - start
print('dur', dur, dur / n)
print(out)

In [ ]:
model = M12dout(z_out=35, nnum=19)

lf2c = Lightfield2Channel(nnum=19)

tgt = numpy.array(y[0])[0]
print('tgt', tgt.shape)

ipt = numpy.array(x[0])[0]
print('ipt', ipt.shape)
ipt = lf2c.tensor_function(ipt)[None, ...]
print('ipt', ipt.shape)
ipt = torch.from_numpy(ipt)
pred = model(ipt)

In [ ]:
for xx, yy in zip(x, y):
    print(xx.shape, yy.shape)